In [1]:
import scipy.io as sio
import numpy as np
import torch
import os
from os import listdir
from os.path import isfile, join
from torch import nn, optim
import torchvision
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import numpy as np 
import math
import pandas as pd
import cmath
import scipy.io as sio
import numpy as np
import torch
from torch import nn, optim
import torchvision
from torch.utils.data import Dataset, DataLoader
import numpy as np 
import math
import pandas as pd
import cmath

#from unet import UNet
# from auto import encoder, decoder

from collections import OrderedDict
from torch.utils.data.sampler import SubsetRandomSampler
from torch.autograd import Variable
#==========================================================================
# For Plotting loss graph
# Bokeh
from bokeh.io import curdoc
from bokeh.layouts import column
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure

from functools import partial
from threading import Thread
from tornado import gen
# from AttRCNN_UNet import Att_R2U
import logging
logging.basicConfig(format='%(asctime)s - %(message)s', level=logging.INFO)
import sys
# from dataloader import norm
import csv
from sklearn.model_selection import train_test_split
from sys import getsizeof
# import wandb
# import logging
# logging.basicConfig(format='%(asctime)s - %(message)s', level=logging.INFO)

In [24]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=(3,3), stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=(3,3), stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = F.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=181):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=(1,3), stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=1)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=1)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=1)
        self.linear = nn.Linear(25, num_classes)
        self.adp_pool = nn.AdaptiveMaxPool2d((4, 25))
        #self.flat = nn.Conv2d(in_channels=512, out_channels=1, kernel_size=1, stride=1)
        self.flat = nn.Sequential(nn.Conv2d(in_channels=512, out_channels=1, kernel_size=1, stride=1))

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        # print("Input ==> ", x.size())
        out = F.relu(self.bn1(self.conv1(x)))
        # print("F.relu(self.bn1(self.conv1(x))) ==> ", x.size())
        out = self.layer1(out)
        # print("layer1 ==> ", x.size())
        out = self.layer2(out)
        # print("layer2 ==> ", x.size())
        out = self.layer3(out)
        # print("layer3 ==> ", x.size())
        out = self.layer4(out)
#         print("layer4 ==>", out.size())
#         out = self.adp_pool(out)
        out = F.avg_pool2d(out, (3, 4))
#         print("avg_pool2d ===>", out.size())
        # out = out.view(out.size(0), -1)
        # print("out.view ===>", out.size())
        out = self.linear(out)
        out = self.flat(out)
#         print("Out ===>", out.size())
        return out


def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])


def ResNet34():
    return ResNet(BasicBlock, [3, 4, 6, 3])

def ResNet101():
    return ResNet(BasicBlock, [3, 4, 23, 3])


if __name__ == "__main__":
    image = torch.rand(1, 3, 10, 100)
    model = ResNet34()
    model(image)


In [2]:
train_dataset_list = []
test_dataset_list = []

In [1]:
df1  = sio.loadmat("../datasets/DOA4/SNR_NS_0_4_100000.mat")
df2  = sio.loadmat("../datasets/DOA4/SNR_NS_10_4_100000.mat")
df3  = sio.loadmat("../datasets/DOA4/SNR_NS_20_4_100000.mat")
df4  = sio.loadmat("../datasets/DOA4/SNR_NS_30_4_100000.mat")
df5  = sio.loadmat("../datasets/DOA4/SNR_NS_40_4_100000.mat")
df = [df1, df2, df3, df4, df5]

NameError: name 'sio' is not defined

In [4]:
new_train_dataset_list = []
new_test_dataset_list = []
train_dataset_list_label = []
test_dataset_list_label = []
def create_dataset(df):
    data = np.transpose(df['NS_data'], (2, 0, 1))
    label = df['DOA']
    X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.15, random_state=42)
    new_train_dataset_list.extend(X_train)
    new_test_dataset_list.append(X_test.tolist())
    train_dataset_list_label.extend(y_train)
    test_dataset_list_label.append(y_test.tolist())
    
for file in df:
    create_dataset(file)

In [5]:
del df
del df1
del df2
del df3
del df4
del df5

In [6]:
for idx, data in enumerate(new_train_dataset_list):
    new = np.zeros((3, 8, 100))
    for j in range(0, data.shape[0]):
        for k in range(0, data.shape[1]):
            new[0][j][k] = data[j][k].real
            new[1][j][k] = data[j][k].imag
            new[2][j][k] = cmath.phase(data[j][k])
    new_train_dataset_list[idx] = new
    
for idx, data in enumerate(new_test_dataset_list):
    for i, ndata in enumerate(data):
        new = np.zeros((3, 8, 100))
        for j in range(0, 8):
            for k in range(0, 100):
                new[0][j][k] = ndata[j][k].real
                new[1][j][k] = ndata[j][k].imag
                new[2][j][k] = cmath.phase(ndata[j][k])
        new_test_dataset_list[idx][i] = new

In [7]:
def get_data(train, test, batch_size, train_bool=True):
    class DOA_dataset(Dataset):
        def __init__(self, train, test):
            self.x = torch.from_numpy(np.array(train))
            self.y = torch.from_numpy(np.asarray(test))
            self.n_sample = len(self.y)
        def __getitem__(self, index):
            return self.x[index], self.y[index]
        def __len__(self):
            return self.n_sample

    dataset = DOA_dataset(train, test)

    loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=train_bool)
    return loader


In [8]:
doa2_train_loader = get_data(new_train_dataset_list, train_dataset_list_label, 64, True)

In [9]:
del new_train_dataset_list
del train_dataset_list_label

In [10]:
test_dataset_list = []
for idx, data in enumerate(new_test_dataset_list):
    print(type(data))
    f = get_data(data, test_dataset_list_label[idx], 128, False)
    test_dataset_list.append(f)

<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>


In [11]:
del new_test_dataset_list
del new

In [12]:
acc_res = {
    "training": [],
    0: [],
    10: [],
    20: [],
    30: [],
    40: []
}
loss_res = {
    "training": [],
    0: [],
    10: [],
    20: [],
    30: [],
    40: []
}
mae_res = {
    "training": [],
    0: [],
    10: [],
    20: [],
    30: [],
    40: []
}

In [25]:
weights = "doa_weights/NS/DOA_4_model.pth"

classification_model = torch.load(weights)
# pre_model
# for prams in classification_model.parameters():
#       prams.requires_grad = False

In [26]:
criterion = nn.CrossEntropyLoss()

if torch.cuda.is_available():
	print(torch.cuda.get_device_name(0))
	classification_model = classification_model.cuda()
	optimizer = optim.AdamW(classification_model.parameters(), lr=0.0001, weight_decay=1e-5)
	criterion = criterion.cuda()

GeForce GTX 1080 Ti


In [27]:
def train():
    print("Training Starts !!!!!!!")
    # Validation for each SNR value
    classification_model.eval()
    total_valdation_loss = 0
    for val_data in range(0, len(test_dataset_list)):

        validation_loss = 0
        validation_acc = 0
        validation_mae = 0.0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for features, labels in test_dataset_list[val_data]:
                features, labels = Variable(features.cuda()), Variable(labels.cuda())
                enn = classification_model(features.float())
                auto_outputs = torch.transpose(enn, 2, 3)
                auto_outputs = torch.reshape(auto_outputs, (auto_outputs.shape[0], 181, 4))
                loss = criterion(auto_outputs.cuda(), labels.type(torch.LongTensor).cuda())

                _, pred = torch.max(auto_outputs, 1)
                val_total+= labels.reshape(-1).size(0)
                val_correct+=(pred.reshape(-1).cuda() == labels.reshape(-1)).sum().item()
                validation_loss += loss.item()
                validation_mae += torch.abs(pred.reshape(-1).cuda() - labels.reshape(-1)).sum().item()

            loss_res[10*val_data].append(validation_loss/len(test_dataset_list[val_data]))
            acc_res[10*val_data].append((100*(val_correct/val_total)))
            mae_res[10*val_data].append(validation_mae/(128*len(test_dataset_list[val_data])*181))

            print('SNR [{}dB], Validation Loss: {:.4f}, Validation Accuracy: {:.4f}, Validation MAE: {}'
                  .format(val_data*10, validation_loss/len(test_dataset_list[val_data]), (100*(val_correct/val_total)), validation_mae/(128*len(test_dataset_list[val_data])*181)))

train()
print("Training Complete")

Training Starts !!!!!!!


RuntimeError: mat1 dim 1 must match mat2 dim 0

In [25]:
# preTrained =  ResNet34()
# weights = "doa_weights/NS/DOA_4_best_model.pth"

# pre_model = torch.load(weights)

# preTrained.load_state_dict(pre_model, strict=False)

# for prams in preTrained.parameters():
#       prams.requires_grad = False

In [9]:
from resnet34 import ResNet34, ResNet18, ResNet101
# from resnext import Inceptionv4
res_model = ResNet34()
# model = res_model

In [10]:
if __name__ == "__main__":
    image = torch.rand(1, 3, 8, 100)
    model = res_model()
    print(model(image).size())


TypeError: forward() missing 1 required positional argument: 'x'

In [6]:
# model

In [7]:
torch.save( model, weights)

In [ ]:
model.eval()
total_valdation_loss = 0
for val_data in range(0, len(test_dataset_list)):
    
    validation_loss = 0
    validation_acc = 0
    validation_mae = 0.0
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for features, labels in test_dataset_list[val_data]:
            features, labels = Variable(features.cuda()), Variable(labels.cuda())
            enn = classification_model(features.float())
            auto_outputs = torch.transpose(enn, 2, 3)
            auto_outputs = torch.reshape(auto_outputs, (auto_outputs.shape[0], 181, 1))
            loss = criterion(auto_outputs.cuda(), labels.type(torch.LongTensor).cuda())

            _, pred = torch.max(auto_outputs, 1)
            val_total+= labels.reshape(-1).size(0)
            # print("######### pred ###########")
            # print(pred[0])
            # print("######### label ###########")
            # print(labels[0])
            val_correct+=(pred.reshape(-1).cuda() == labels.reshape(-1)).sum().item()
            validation_loss += loss.item()
            validation_mae += torch.abs(pred.reshape(-1).cuda() - labels.reshape(-1)).sum().item()

        loss_res[10*val_data].append(validation_loss/len(test_dataset_list[val_data]))
        acc_res[10*val_data].append((100*(val_correct/val_total)))
    print(val_correct, val_total)
    print(val_data*10, "dB SNR is validated")
    print('SNR [{}dB], Validation Loss: {:.4f}, Validation Accuracy: {:.4f}'
              .format(val_data*10, validation_loss/len(doa2_train_loader), (100*(val_correct/val_total))))